<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Neural_Architecture_Search_(NAS).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install autokeras

In [ ]:
pip install tf-keras --upgrade

In [ ]:
pip install tensorflow-gpu

In [ ]:
import autokeras as ak
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Load your dataset (example using CIFAR-10 dataset)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Normalize the dataset
x_train, x_test = x_train / 255.0, x_test / 255.0

# Split the dataset further into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# Initialize the AutoKeras ImageClassifier
clf = ak.ImageClassifier(overwrite=True, max_trials=10)

# Train the model
clf.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val))

# Evaluate the model
results = clf.evaluate(x_test, y_test)
print(f"Test accuracy: {results[1]}")

# Make predictions
predictions = clf.predict(x_test)
print(f"Predictions: {predictions}")